In [33]:
import numpy as np
#Sharing Data
%store -r tf
%store -r R_pd1
trainset = R_pd1
testset= tf

In [41]:
trainset=trainset.rename(columns={"userID": "uid", "itemID": "mid", "rating": "rat"})

In [45]:
test=trainset
test.uid=test.uid-1
test.mid=test.mid-1

In [54]:
test.rat[1]

5

In [59]:
import math
import random
import time

class SvdMatrix:
    """
    trainfile -> name of file to train data against
    nusers -> number of users in dataset
    nmovies -> number of movies in dataset
    r -> rank of approximation (for U and V)
    lrate -> learning rate
    regularizer -> regularizer
    typefile -> 0 if for smaller MovieLens dataset
                1 if for medium or larger MovieLens dataset
    """
    def __init__(self, data, r=30, lrate=0.035, regularizer=0.01):
        self.trainrats = data#[]
       #self.testrats = []
        #self.data = data
        self.users = data.uid
        self.movies = data.mid
        self.rating = data.rat
        # get average rating
        avg = self.averagerating()

        self.r = r
        self.lrate = lrate
        self.regularizer = regularizer
        self.minimprov = 0.001
        self.maxepochs = 30

         # set initial values in U, V using square root
        # of average/rank
        initval = math.sqrt(avg/r)

        # U matrix
        self.U = [[initval]*r for i in self.users]
        # V matrix -- easier to store and compute than V^T
        self.V = [[initval]*r for i in self.movies]

    #avant
    def averagerating(self):
        avg = 0
        n = 0
        for i in range(len(self.trainrats)):
            #avg += self.trainrats.rat[i]
            avg += self.trainrats['rat'].iloc[i]
            n += 1
        return float(avg/n)
    #apres



if __name__ == "__main__":
    #========= test SvdMatrix class on smallest MovieLENS dataset =========
    init = time.time()
    svd = SvdMatrix(test)

In [ ]:
  #AVANT

    """
    Returns the dot product of v1 and v2
    """
    def dotproduct(self, v1, v2):
        return sum([v1[i]*v2[i] for i in range(len(v1))])

    """
    Returns the estimated rating corresponding to userid for movieid
    Ensures returns rating is in range [1,5]
    """
    def calcrating(self, uid, mid):
        p = self.dotproduct(self.U[uid], self.V[mid])
        if p > 5:
            p = 5
        elif p < 1:
            p = 1
        return p

    """
    Returns the average rating of the entire dataset
    """

In [ ]:
#APRES
"""
    Predicts the estimated rating for user with id i
    for movie with id j
    """
    def predict(self, i, j):
        return self.calcrating(i, j)

    """
    Trains the kth column in U and the kth row in
    V^T
    See docs for more details.
    """
    def train(self, k):
        sse = 0.0
        n = 0
        for i in range(len(self.trainrats)):
            # get current rating
            crating = self.trainrats[i]
            err = crating.rat - self.predict(crating.uid, crating.mid)
            sse += err**2
            n += 1

            uTemp = self.U[crating.uid][k]
            vTemp = self.V[crating.mid][k]

            self.U[crating.uid][k] += self.lrate * (err*vTemp - self.regularizer*uTemp)
            self.V[crating.mid][k] += self.lrate * (err*uTemp - self.regularizer*vTemp)
        return math.sqrt(sse/n)

    """
    Trains the entire U matrix and the entire V (and V^T) matrix
    """
    def trainratings(self):
        # stub -- initial train error
        oldtrainerr = 1000000.0

        for k in range(self.r):
            print("k=", k)
            for epoch in range(self.maxepochs):
                trainerr = self.train(k)

                # check if train error is still changing
                if abs(oldtrainerr-trainerr) < self.minimprov:
                    break
                oldtrainerr = trainerr
                print("epoch=", epoch, "; trainerr=", trainerr)

    """
    Calculates the RMSE using between arr
    and the estimated values in (U * V^T)
    """
    def calcrmse(self, arr):
        nusers = self.nusers
        nmovies = self.nmovies
        sse = 0.0
        total = 0
        for i in range(len(arr)):
            crating = arr[i]
            sse += (crating.rat - self.calcrating(crating.uid, crating.mid))**2
            total += 1
        return math.sqrt(sse/total)

In [ ]:
    svd.trainratings()
    print("rmsetrain: ", svd.calcrmse(svd.trainrats))
    print("rmsetest: ", svd.calcrmse(svd.testrats))
    print("time: ", time.time()-init)

In [60]:
# Daniel Alabi and Cody Wang
# ======================================
# SvdMatrix:
# generates matrices U and V such that
# U * V^T closely approximates
# the original matrix (in this case, the utility
# matrix M)
# =======================================


import math
import random
import time

"""
Rating class.
Store every rating associated with a particular
userid and movieid.
================Optimization======================
"""
class Rating:
    def __init__(self, userid, movieid, rating):
        # to accomodate zero-indexing for matrices
        self.uid = userid-1
        self.mid = movieid-1

        self.rat = rating


class SvdMatrix:
    """
    trainfile -> name of file to train data against
    nusers -> number of users in dataset
    nmovies -> number of movies in dataset
    r -> rank of approximation (for U and V)
    lrate -> learning rate
    regularizer -> regularizer
    typefile -> 0 if for smaller MovieLens dataset
                1 if for medium or larger MovieLens dataset
    """
    def __init__(self, trainfile, nusers, nmovies, r=30, lrate=0.035, regularizer=0.01, typefile=0):
        self.trainrats = []
        self.testrats = []

        self.nusers = nusers
        self.nmovies = nmovies

        if typefile == 0:
            self.readtrainsmaller(trainfile)
        elif typefile == 1:
            self.readtrainlarger(trainfile)

        # get average rating
        avg = self.averagerating()
        # set initial values in U, V using square root
        # of average/rank
        initval = math.sqrt(avg/r)

        # U matrix
        self.U = [[initval]*r for i in range(nusers)]
        # V matrix -- easier to store and compute than V^T
        self.V = [[initval]*r for i in range(nmovies)]

        self.r = r
        self.lrate = lrate
        self.regularizer = regularizer
        self.minimprov = 0.001
        self.maxepochs = 30

    """
    Returns the dot product of v1 and v2
    """
    def dotproduct(self, v1, v2):
        return sum([v1[i]*v2[i] for i in range(len(v1))])

    """
    Returns the estimated rating corresponding to userid for movieid
    Ensures returns rating is in range [1,5]
    """
    def calcrating(self, uid, mid):
        p = self.dotproduct(self.U[uid], self.V[mid])
        if p > 5:
            p = 5
        elif p < 1:
            p = 1
        return p

    """
    Returns the average rating of the entire dataset
    """
    def averagerating(self):
        avg = 0
        n = 0
        for i in range(len(self.trainrats)):
            avg += self.trainrats[i].rat
            n += 1
        return float(avg/n)

    """
    Predicts the estimated rating for user with id i
    for movie with id j
    """
    def predict(self, i, j):
        return self.calcrating(i, j)

    """
    Trains the kth column in U and the kth row in
    V^T
    See docs for more details.
    """
    def train(self, k):
        sse = 0.0
        n = 0
        for i in range(len(self.trainrats)):
            # get current rating
            crating = self.trainrats[i]
            err = crating.rat - self.predict(crating.uid, crating.mid)
            sse += err**2
            n += 1

            uTemp = self.U[crating.uid][k]
            vTemp = self.V[crating.mid][k]

            self.U[crating.uid][k] += self.lrate * (err*vTemp - self.regularizer*uTemp)
            self.V[crating.mid][k] += self.lrate * (err*uTemp - self.regularizer*vTemp)
        return math.sqrt(sse/n)

    """
    Trains the entire U matrix and the entire V (and V^T) matrix
    """
    def trainratings(self):
        # stub -- initial train error
        oldtrainerr = 1000000.0

        for k in range(self.r):
            print("k=", k)
            for epoch in range(self.maxepochs):
                trainerr = self.train(k)

                # check if train error is still changing
                if abs(oldtrainerr-trainerr) < self.minimprov:
                    break
                oldtrainerr = trainerr
                print("epoch=", epoch, "; trainerr=", trainerr)

    """
    Calculates the RMSE using between arr
    and the estimated values in (U * V^T)
    """
    def calcrmse(self, arr):
        nusers = self.nusers
        nmovies = self.nmovies
        sse = 0.0
        total = 0
        for i in range(len(arr)):
            crating = arr[i]
            sse += (crating.rat - self.calcrating(crating.uid, crating.mid))**2
            total += 1
        return math.sqrt(sse/total)

    """
    Read in the ratings from fname and put in arr
    Use splitter as delimiter in fname
    """
    def readinratings(self, fname, arr, splitter="\t"):
        f = open(fname)

        for line in f:
            newline = [int(each) for each in line.split(splitter)]
            userid, movieid, rating = newline[0], newline[1], newline[2]
            arr.append(Rating(userid, movieid, rating))

        arr = sorted(arr, key=lambda rating: (rating.uid, rating.mid))
        return len(arr)

    """
    Read in the smaller train dataset
    """
    def readtrainsmaller(self, fname):
        return self.readinratings(fname, self.trainrats, splitter="\t")

    """
    Read in the large train dataset
    """
    def readtrainlarger(self, fname):
        return self.readinratings(fname, self.trainrats, splitter="::")

    """
    Read in the smaller test dataset
    """
    def readtestsmaller(self, fname):
        return self.readinratings(fname, self.testrats, splitter="\t")

    """
    Read in the larger test dataset
    """
    def readtestlarger(self, fname):
        return self.readinratings(fname, self.testrats, splitter="::")


if __name__ == "__main__":
    #========= test SvdMatrix class on smallest MovieLENS dataset =========
    init = time.time()
    svd = SvdMatrix("ua.base", 943, 1682)
    svd.trainratings()
    print("rmsetrain: ", svd.calcrmse(svd.trainrats))
    svd.readtestsmaller("ua.test")
    print("rmsetest: ", svd.calcrmse(svd.testrats))
    print("time: ", time.time()-init)

k= 0
epoch= 0 ; trainerr= 1.0432311376555137
epoch= 1 ; trainerr= 1.022065752726435
epoch= 2 ; trainerr= 1.015252228247634
epoch= 3 ; trainerr= 1.0100264253173465
epoch= 4 ; trainerr= 1.0056150198969338
epoch= 5 ; trainerr= 1.0020928548317705
epoch= 6 ; trainerr= 0.9993244945716312
epoch= 7 ; trainerr= 0.9971475438583652
epoch= 8 ; trainerr= 0.9953896929132656
epoch= 9 ; trainerr= 0.9939424745347585
epoch= 10 ; trainerr= 0.992721839939337
epoch= 11 ; trainerr= 0.9916788424625941
k= 1
epoch= 0 ; trainerr= 0.9576460274585444
epoch= 1 ; trainerr= 0.9298693334137687
epoch= 2 ; trainerr= 0.9239615664078827
epoch= 3 ; trainerr= 0.9217576789917138
epoch= 4 ; trainerr= 0.9204936740724455
k= 2
epoch= 0 ; trainerr= 0.909394992873761
epoch= 1 ; trainerr= 0.9031912471454353
epoch= 2 ; trainerr= 0.8994533730580375
epoch= 3 ; trainerr= 0.8975216073989023
epoch= 4 ; trainerr= 0.8964156355468492
k= 3
epoch= 0 ; trainerr= 0.8905722722953182
epoch= 1 ; trainerr= 0.8869459672069178
epoch= 2 ; trainerr= 0

In [62]:
print(svd.U)
print(svd.V)


[[1.3724816042404593, -0.2131930843204195, 0.310571783432505, 0.45978427837497626, 0.46340488822157555, 0.24132861984653237, 0.457171320285497, 0.3221600685574911, 0.3312723604013475, 0.34413654656325204, 0.34307127335636584, 0.3424344277582587, 0.34211110778370046, 0.34194071711653123, 0.3418476018388837, 0.34179552353262677, 0.34176671454857194, 0.34169319757190253, 0.3416600408480719, 0.3416501470096662, 0.3416508683343464, 0.3416567332426363, 0.34166527906039657, 0.3416753153110514, 0.3416862579299963, 0.34169774706948286, 0.3417095474466458, 0.34172149398590074, 0.3417334775285972, 0.34174542178647094], [1.0145950494513627, 0.11715847661573367, 0.1758601002106702, 0.36400045902830636, 0.48375124473524655, 0.3416479106265537, 0.3384968447285195, 0.34618232353385786, 0.3451311823821339, 0.343202893365051, 0.34309720221511664, 0.3428552523585741, 0.34262608094502595, 0.34243115566691856, 0.34226940772233305, 0.3421360549361281, 0.3420263498526104, 0.34193637013434824, 0.3418626549300